### Steps:

1. Sing up on https://www.comet.ml
2. See quickstart guide
3. Run this example (which is Basic MNIST Example from https://github.com/pytorch/examples/tree/master/mnist)
4. Compare our [result](https://www.comet.ml/bamasa/general/1b6c77bb08654fcea9a9ae2a4417cc00) with your one.

In [2]:
# Basic MNIST Example from https://github.com/pytorch/examples/tree/master/mnist.

# Import comet_ml in the top of your file
from comet_ml import Experiment
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time

In [4]:
# Create an experiment
# api_key could be found on https://www.comet.ml
experiment = Experiment(api_key=<your api key>,
                        project_name="general", workspace=<your workspace>)

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/bamasa/general/16e775b8a9bd4db39724bd718aef2f40
COMET INFO: ----------------------------
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/bamasa/general/1b6c77bb08654fcea9a9ae2a4417cc00



In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, print_out=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if print_out:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [14]:
# Report any information you need by:
hyper_params = {
    "batch_size": 32,
    "test_batch_size": 32,
    "epochs": 10,
    "lr": 0.01,
    "momentum": 0.5,
    "no_cuda": False,
    "seed": 1,
    "log_interval": 50,
    "save_model": False,
}
experiment.log_parameters(hyper_params)

In [15]:
start_time = time.time()

use_cuda = not hyper_params["no_cuda"] and torch.cuda.is_available()
experiment.log_parameter("use_cuda", use_cuda)

torch.manual_seed(hyper_params["seed"])

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=hyper_params["batch_size"], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=hyper_params["test_batch_size"], shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=hyper_params["lr"], momentum=hyper_params["momentum"])

print("###start training")

for epoch in range(1, hyper_params["epochs"] + 1):
    train(hyper_params["log_interval"], model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
test(model, device, test_loader, print_out=True)

if (hyper_params["save_model"]):
    torch.save(model.state_dict(), "mnist_cnn.pt")

print("###end training, time: {}".format(time.time() - start_time))

###start training
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301286
Train Epoch: 1 [1600/60000 (3%)]	Loss: 1.387788
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.640893
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.554110
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.155472
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.322598
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.169638
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.230502
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.398340
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.075587
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.206865
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.284769
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.199208
Train Epoch: 1 [20800/60000 (35%)]	Loss: 0.218565
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.116803
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.097971
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.218402
Train Epoch: 1 [27200/60000 (45%)]	Loss: 0.147269
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.050427
Train Epoch: 1 [30400/60000 (51%)]	Loss: 0.135

Train Epoch: 5 [20800/60000 (35%)]	Loss: 0.095415
Train Epoch: 5 [22400/60000 (37%)]	Loss: 0.001476
Train Epoch: 5 [24000/60000 (40%)]	Loss: 0.000638
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.022065
Train Epoch: 5 [27200/60000 (45%)]	Loss: 0.010565
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.002569
Train Epoch: 5 [30400/60000 (51%)]	Loss: 0.003642
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.018185
Train Epoch: 5 [33600/60000 (56%)]	Loss: 0.008090
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.014817
Train Epoch: 5 [36800/60000 (61%)]	Loss: 0.003802
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.031541
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.012959
Train Epoch: 5 [41600/60000 (69%)]	Loss: 0.004818
Train Epoch: 5 [43200/60000 (72%)]	Loss: 0.036366
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.002965
Train Epoch: 5 [46400/60000 (77%)]	Loss: 0.178241
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.015986
Train Epoch: 5 [49600/60000 (83%)]	Loss: 0.016409
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.014469


Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.001950
Train Epoch: 9 [43200/60000 (72%)]	Loss: 0.006497
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.002673
Train Epoch: 9 [46400/60000 (77%)]	Loss: 0.024171
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.000118
Train Epoch: 9 [49600/60000 (83%)]	Loss: 0.009322
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.005642
Train Epoch: 9 [52800/60000 (88%)]	Loss: 0.015002
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.000084
Train Epoch: 9 [56000/60000 (93%)]	Loss: 0.095492
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.012699
Train Epoch: 9 [59200/60000 (99%)]	Loss: 0.004073
Train Epoch: 10 [0/60000 (0%)]	Loss: 0.135136
Train Epoch: 10 [1600/60000 (3%)]	Loss: 0.002625
Train Epoch: 10 [3200/60000 (5%)]	Loss: 0.003352
Train Epoch: 10 [4800/60000 (8%)]	Loss: 0.056217
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.007494
Train Epoch: 10 [8000/60000 (13%)]	Loss: 0.000526
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.005748
Train Epoch: 10 [11200/60000 (19%)]	Loss: 0.020457
Train 